In [1]:
!git clone https://github.com/hukim1112/MLDL.git

fatal: destination path 'MLDL' already exists and is not an empty directory.


In [2]:
import tensorflow as tf
import json, os, sys, time
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [3]:
sys.path.append("/content/MLDL/object_detection")

In [4]:
import anchor, losses, manage_checkpoint

In [30]:
config = {
    "exp_desc" : "perfection", #실험명
    "sub_desc" : "exp1", #하위실험명
    "image_dir" : "/content/images", #이미지경로
    "coco_api" : "/home/dan/prj/PythonAPI",
    "annotation_dir" : "/content/", #어노테이션 경로
    "label_set" : ["head","helmet"], # 분류할 오브젝트 집합
    "input_shape" : [160, 160, 3], #모델입력영상 크기
    "arch" : "ssd160",
    "backbone" : "mobilenet",
    "anchor_param" : {"ratios": [[2], [2, 3], [2, 3], [2]],
                           "scales": [0.1, 0.3, 0.6, 0.9, 1.05],
                           "fm_sizes": [10, 5, 3, 1],
                           "image_size": 160}, #anchor parameters
    "ckpt":
    {
        "save_type" : "best",
        "max_to_keep" : 10,
        "pretrained_type" : "init",
        "model_path" : None
    },
    "inference_mode" : "train", # train or mAP
    "train" :
    {
        "num_examples" : -1,
        "batch_size" : 16,
        "augmentation" : True,
        "random_crop_rate" : 0.3,
        "neg_ratio" : 3,
        "initial_lr" : 1e-3,
        "momentum" : 0.9,
        "weight_decay" : 5e-5,
        "num_epochs" : 300
    },
    "val":
    {
        "num_examples" : -1,
        "batch_size" : 64
    },
    "test":
    {
        "num_examples" : -1,
        "batch_size" : 1
    }

}
num_classes = len(config['label_set'])
log_dir = os.path.join('logs', 'perfection', 'exp1')
# if os.path.isdir(ckpt_dir):
#     raise ValueError("checkpoint directory exists. checkout your experiment name in configure file.")
# if os.path.isdir(log_dir):
#     raise ValueError("log directory exists. checkout your experiment name in configure file.")


# Dataset load

In [6]:
from pycocotools.coco import COCO
# test COCO api can load my json file
json = COCO("/content/GDUT_HWD.json")

In [16]:
!ls images.zip
!unzip images.zip

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [33]:
from coco import Dataset
ds_obj = Dataset(config, COCO)

In [34]:
batch_generator, train_length = ds_obj.load_data_generator('train', config)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [35]:
for i, (_, imgs, gt_confs, gt_locs) in enumerate(batch_generator.take(1)):
    print(imgs.shape, gt_confs.shape, gt_locs.shape)

(16, 160, 160, 3) (16, 608) (16, 608, 4)


# Model load

In [37]:
model = tf.keras.models.load_model("ssd160-mobilenet.h5")

In [38]:
model.layers[1].trainable = False
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
model_13 (Functional)           (None, 10, 10, 576)  616256      input_11[0][0]                   
__________________________________________________________________________________________________
sequential_12 (Sequential)      (None, 5, 5, 512)    569344      model_13[0][0]                   
__________________________________________________________________________________________________
sequential_13 (Sequential)      (None, 3, 3, 512)    536576      sequential_12[0][0]              
___________________________________________________________________________________________

In [39]:
model(imgs)

[<tf.Tensor: shape=(16, 608, 3), dtype=float32, numpy=
 array([[[ 7.85903931e-01, -6.54461861e-01,  5.73911130e-01],
         [-9.75549400e-01, -8.22570026e-02,  1.09906983e+00],
         [ 2.65728593e-01,  3.77192736e-01,  1.92717433e-01],
         ...,
         [-3.61195873e-07, -4.01300122e-06, -1.50403068e-07],
         [ 1.45957051e-06,  2.76056653e-06,  8.38467713e-06],
         [-6.34445632e-06, -2.42983333e-06, -3.37804704e-06]],
 
        [[ 2.97688603e-01, -9.72911298e-01,  4.49492902e-01],
         [-1.11551595e+00,  1.35816336e-01,  3.16954851e-01],
         [ 1.76708102e-02,  2.03252077e-01,  2.05680132e-02],
         ...,
         [ 5.57828253e-06, -3.94437120e-06, -9.78202706e-07],
         [ 2.60399634e-06,  2.86339059e-06,  1.08349823e-05],
         [-6.21935396e-06, -1.16752813e-06, -4.16094736e-06]],
 
        [[ 3.50150347e-01, -4.94881630e-01,  1.28381565e-01],
         [-3.65332305e-01,  2.40958273e-01, -5.23854792e-02],
         [ 1.62983239e-01,  3.74725580e-01,

# Training loop

In [40]:
@tf.function
def train_step(imgs, gt_confs, gt_locs, ssd, criterion, optimizer, config):
    with tf.GradientTape() as tape:
        confs, locs = ssd(imgs)

        conf_loss, loc_loss = criterion(
            confs, locs, gt_confs, gt_locs)

        loss = conf_loss + loc_loss
        l2_loss = [tf.nn.l2_loss(t) for t in ssd.trainable_variables]
        l2_loss = config['train']['weight_decay'] * tf.math.reduce_sum(l2_loss)
        loss += l2_loss

    gradients = tape.gradient(loss, ssd.trainable_variables)
    optimizer.apply_gradients(zip(gradients, ssd.trainable_variables))

    return loss, conf_loss, loc_loss, l2_loss

criterion = losses.create_losses(config['train']['neg_ratio'], num_classes)

In [41]:
summary_writer = tf.summary.create_file_writer(log_dir)

t_loss = tf.metrics.Mean(name='train_loss')
t_conf_loss = tf.metrics.Mean(name='train_conf_loss')
t_loc_loss = tf.metrics.Mean(name='train_loc_loss')
# v_loss = tf.metrics.Mean(name='val_loss')
# v_conf_loss = tf.metrics.Mean(name='val_conf_loss')
# v_loc_loss = tf.metrics.Mean(name='val_loc_loss')

learning_rate = 1E-3
EPOCH = 50
START = 0
optimizer = tf.keras.optimizers.Adam(learning_rate)
for epoch in range(START+1, START+EPOCH+1):
    start = time.time()
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(batch_generator):
        imgs = preprocess_input(imgs)
        loss, conf_loss, loc_loss, l2_loss = train_step(imgs, gt_confs, gt_locs, model, criterion, optimizer, config)
        t_loss(loss)
        t_conf_loss(conf_loss)
        t_loc_loss(loc_loss)
        
        print("Epoch {} iteration {} loss : {}".format(epoch, i, t_loss.result()))
    
    '''
    you can add the validation part
    
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(val_generator):
        imgs = preprocess_input(imgs)
        val_confs, val_locs = model(imgs)
        val_conf_loss, val_loc_loss = criterion(val_confs, val_locs, gt_confs, gt_locs)
        v_loss(val_conf_loss+val_loc_loss)
        v_conf_loss(val_conf_loss)
        v_loc_loss(val_loc_loss)
    '''
    # Save checkpoint with a strategy.
    if epoch%10 == 0:
        model.save("{}.h5".format(epoch))

    with summary_writer.as_default():
        tf.summary.scalar('train_loss', t_loss.result(), step=epoch)
        tf.summary.scalar('train_conf_loss', t_conf_loss.result(), step=epoch)
        tf.summary.scalar('train_loc_loss', t_loc_loss.result(), step=epoch)
        #tf.summary.scalar('val_loss', v_loss.result(), step=epoch)
        #tf.summary.scalar('val_conf_loss', v_conf_loss.result(), step=epoch)
        #tf.summary.scalar('val_loc_loss', v_loc_loss.result(), step=epoch)
        tf.summary.scalar('learning_rate', learning_rate, step=epoch)
        
    t_loss.reset_states()
    t_conf_loss.reset_states()
    t_loc_loss.reset_states()
    #v_loss.reset_states()
    #v_conf_loss.reset_states()
    #v_loc_loss.reset_states()   

Epoch 1 iteration 0 loss : 12.138065338134766
Epoch 1 iteration 1 loss : 10.540584564208984
Epoch 1 iteration 2 loss : 9.416193962097168
Epoch 1 iteration 3 loss : 9.12147045135498
Epoch 1 iteration 4 loss : 8.641573905944824
Epoch 1 iteration 5 loss : 8.301690101623535
Epoch 1 iteration 6 loss : 8.065224647521973
Epoch 1 iteration 7 loss : 7.990416526794434
Epoch 1 iteration 8 loss : 7.959733963012695
Epoch 1 iteration 9 loss : 7.814990997314453
Epoch 2 iteration 0 loss : 5.855602741241455
Epoch 2 iteration 1 loss : 6.279521465301514
Epoch 2 iteration 2 loss : 6.455644130706787
Epoch 2 iteration 3 loss : 6.094285011291504
Epoch 2 iteration 4 loss : 5.8418989181518555
Epoch 2 iteration 5 loss : 5.655609130859375
Epoch 2 iteration 6 loss : 5.518134117126465
Epoch 2 iteration 7 loss : 5.443604469299316
Epoch 2 iteration 8 loss : 5.35369873046875
Epoch 2 iteration 9 loss : 5.297483444213867
Epoch 3 iteration 0 loss : 3.9310946464538574
Epoch 3 iteration 1 loss : 4.00526237487793
Epoch 3 i

KeyboardInterrupt: ignored